<a href="https://colab.research.google.com/github/iamRahulB/Pytorch-practice/blob/main/TEXT/LSTM_on_custom_data_with_tfidf_vectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install -q torch==2.2.0 torchtext==0.17.0 --index-url https://download.pytorch.org/whl/cu118

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.7/811.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 104.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [2]:
!pip install -q -U portalocker==2.8.2

In [3]:
import torch
from torch import nn
import torch.optim as optim
from torchtext import datasets
from torch.utils.data import DataLoader, Dataset
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import re


In [4]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
df=pd.read_csv('https://github.com/LawrenceDuan/IMDb-Review-Analysis/raw/refs/heads/master/IMDb_Reviews.csv')

In [6]:
df.head()

,review,sentiment
0,My family and I normally do not watch local mo...,1
1,"Believe it or not, this was at one time the wo...",0
2,"After some internet surfing, I found the ""Home...",0
3,One of the most unheralded great works of anim...,1
4,"It was the Sixties, and anyone with long hair ...",0


In [7]:
stop_words=stopwords.words('english')
lemma=WordNetLemmatizer()
print(f"Lemmatizer Result : {lemma.lemmatize('butterflies')}")
print(f'Stopwords Result: {stop_words}')

Lemmatizer Result : butterfly
Stopwords Result: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor

In [8]:
sample_data=df.sample(4000)
sample_data['sentiment'].value_counts()

,count
sentiment,
1,2003
0,1997


In [9]:
# Upon analysing above data we can see that its not imbalanced dataset as i picked this version on purpose so there will not be any problem ahead

In [10]:
def preprocess(text):
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'[^a-z\s]', '', text, flags=re.IGNORECASE)
    text=[lemma.lemmatize(token.lower()) for token in word_tokenize(text) if token not in stop_words]
    return " ".join(text)

sample_data['cleaned']=sample_data['review'].apply(preprocess)

In [11]:
sample_data.reset_index(inplace=True,drop=True)
sample_data.head(7)

,review,sentiment,cleaned
0,The idea for the movie wasn't too bad: a horro...,0,the idea movie wasnt bad horror film shot corn...
1,"Somebody owes Ang Lee an apology. Actually, a ...",0,somebody owes ang lee apology actually lot peo...
2,This game is very good for the n64. You can sk...,1,this game good n you skate tony hawk bob burnq...
3,B. Kennedy tried to make a sequel by exaggerat...,0,b kennedy tried make sequel exaggerating ampli...
4,"From the very beginning, the political theme o...",0,from beginning political theme film obvious he...
5,i couldn't help but think of behind the mask: ...,0,couldnt help think behind mask rise leslie ver...
6,Such a pretentious and lame attempt to hipness...,0,such pretentious lame attempt hipness diabolic...


In [12]:
#  so this looks good as its not imbalanced dataset

train_data,test_data=train_test_split(sample_data,random_state=42,test_size=0.3,shuffle=True)
pd.DataFrame({"train":train_data['sentiment'].value_counts().to_dict(),"Test" :test_data['sentiment'].value_counts().to_dict()})

,train,Test
1,1407,596
0,1393,604


In [13]:
train_data.head(5)

,review,sentiment,cleaned
3215,An unpleasant woman and an equally unpleasant ...,0,an unpleasant woman equally unpleasant man vio...
3126,"Ah, McBain The character name is immortalized...",1,ah mcbain the character name immortalized fore...
697,I make just one apology for this film: there a...,1,i make one apology film far far many wide angl...
3613,"I, too, found ""Oppenheimer"" to be a brilliant ...",1,i found oppenheimer brilliant series one fines...
2374,For anyone who may not know what a one-actor m...,1,for anyone may know oneactor movie like best e...


In [14]:
tokenizer=get_tokenizer("basic_english")

In [15]:
#  lets create custom iterator from Dataset clas of pytorch

class CustomDataset(Dataset):
    def __init__(self,labels,texts):
        super().__init__()
        self.labels=labels
        self.texts=texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self,idx):
        texts=self.texts[idx]
        labels=self.labels[idx]

        return labels,texts


In [16]:
def get_tokens(texts):
    for text in texts:
        yield tokenizer(text)

In [17]:
# lets build vocab, vocab should be built from iterator also on only train data, as if we passed normal list then vocab from  iterator will not work

vocab=build_vocab_from_iterator(get_tokens(train_data['cleaned'].tolist()),specials=['<unk>'])

vocab.set_default_index(0)

In [18]:
len(vocab)

34633

In [19]:
text_pipeline=lambda x: vocab(tokenizer(x))

label_pipeline=lambda x: int(x)

In [20]:
text_pipeline('hello how are you gefuheu tgrathrhtytytyjtyjtyjtjte') , label_pipeline('434')

([5514, 451, 1790, 167, 0, 0], 434)

So the text pipeline and label pipeline is working great so next step is to convert data into vectorization

1. its good to apply vectorization after splitting the data to avoid the data leakage and test data will not influence bad on the training processs .

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [22]:
tfidf=TfidfVectorizer()

train_tfidf=tfidf.fit_transform(train_data['cleaned'].tolist()).toarray()

test_tfidf=tfidf.transform(test_data['cleaned'].tolist()).toarray()

In [23]:
train_labels=train_data['sentiment'].tolist()
test_labels=test_data['sentiment'].tolist()

In [24]:
train_data['cleaned']

,cleaned
3215,an unpleasant woman equally unpleasant man vio...
3126,ah mcbain the character name immortalized fore...
697,i make one apology film far far many wide angl...
3613,i found oppenheimer brilliant series one fines...
2374,for anyone may know oneactor movie like best e...
...,...
1130,admittedly seeing house wax i assumed would si...
1294,i admit awe sea spent number year navy i somew...
860,i love movie much it always make cry if apprec...
3507,ive already seen spinoffs cartoon the flintsto...


Lets make iterator now to go ahead to genetae data loader


In [25]:
train_iter=CustomDataset(train_labels, train_tfidf)

test_iter=CustomDataset(test_labels,test_tfidf)

In [26]:
len(vocab)

34633

In [27]:
train_tfidf.shape

(2800, 34606)

In [28]:
def collate_batch(batch):
    all_labels=[]
    all_texts=[]
    for labels , texts in batch:
        all_texts.append(texts)
        all_labels.append(label_pipeline(labels))

    return torch.tensor(all_texts,dtype=torch.float),torch.tensor(all_labels,dtype=torch.int64)


In [29]:
train_loader=DataLoader(dataset=train_iter,batch_size=8,shuffle=True,collate_fn=collate_batch)

test_loader=DataLoader(dataset=test_iter,batch_size=8,shuffle=False,collate_fn=collate_batch)

Always remember:
1. Dense vectors : most or all values are non zero in the vectors its called as dense represenattion of vectores.
2. all or most values in the vectors are zeroes are called as sparase matrix or vectors.

If the vectors representation is dense then we dont use embedding layer in neural network.

if the vecotr representation is sparse then we use embedding layer to convert sparse inti dense with embedding layer

----------------------------------------------------------------------------------

### Training and Testing with TF-IDF

#### Fit and Transform on Training Data:
- **Fit**: The `fit` method learns the vocabulary and the IDF values from the training data.
- **Transform**: The `transform` method converts the training data into TF-IDF vectors based on the learned vocabulary and IDF values.

#### Transform on Test Data:
- **Transform**: The `transform` method is used on the test data to convert it into TF-IDF vectors using the same vocabulary and IDF values learned from the training data. This ensures consistency between training and testing.


In [30]:
# lets build the model

class TfidfModel(nn.Module):
    def __init__(self,input_size):
        super().__init__()

        self.lstm=nn.LSTM(input_size=input_size,hidden_size=64,num_layers=1,batch_first=True)   #batch_first--> with this parameter we need to pass the data in which batch size should be at start then seq len

        self.linear=nn.Linear(in_features=64,out_features=1)

    def forward(self,x):
        lstm_out,(hn,cn)=self.lstm(x)
        # print(f"Shape of lstm_out: {lstm_out.shape}")
        x=self.linear(lstm_out[:,-1,:])
        return x

In [31]:
train_tfidf.shape[1]

34606

if we get error too many diemention for index 2 like this then we are not passing the input in expected format so we need to unsquueze the dim

sequence model expects input in `batch_size` , `sequence length`, `input_size`

batch_size=8
sequence len= each time step (in our case its each document tensor)
inpu size= size of the tensor which is 35154

In [32]:
next(iter(train_loader))[0].shape     # this is size of the doc 1 -> 35154,  so its 1 st time step, so we can make it size

<ipython-input-28-6e57b3604305>:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  return torch.tensor(all_texts,dtype=torch.float),torch.tensor(all_labels,dtype=torch.int64)


torch.Size([8, 34606])

In [33]:
input_size=train_tfidf.shape[1]
model=TfidfModel(input_size=input_size)


with torch.inference_mode():
    out=model(next(iter(train_loader))[0].unsqueeze(1))

In [34]:
(out.sigmoid()>0.5).int()

tensor([[1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1]], dtype=torch.int32)

In [35]:
from sklearn.metrics import accuracy_score

In [36]:
from tqdm.auto import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

loss_fn=nn.BCEWithLogitsLoss()
optimizer=torch.optim.Adam(params=model.parameters(),lr=0.001)

# Set the random seed before the loop
torch.manual_seed(42)

# Assuming loss_fn is nn.BCEWithLogitsLoss for binary classification

# Move the model to the GPU
model = model.to(device)

# Training loop
epochs = 5

for epoch in tqdm(range(epochs)):
    train_loss = 0
    train_acc = 0
    model.train()

    for batch, (X_train,y_train) in enumerate(train_loader):
        # Move data to the GPU
        X_train = X_train.to(device)
        y_train = y_train.to(device)

        # Forward pass
        y_logits = model(X_train.unsqueeze(1))

        # Calculate loss
        y_train = y_train.float().unsqueeze(1)
        train_batch_loss = loss_fn(y_logits, y_train)

        # Apply sigmoid and threshold to get binary predictions
        train_probs = torch.sigmoid(y_logits)
        train_batch_preds = (train_probs > 0.5).float()

        # Calculate accuracy using sklearn
        train_batch_acc = accuracy_score(y_train.cpu().numpy(), train_batch_preds.cpu().numpy())

        # Backpropagation and optimization
        optimizer.zero_grad()
        train_batch_loss.backward()
        optimizer.step()

        train_loss += train_batch_loss.item()
        train_acc += train_batch_acc

    model.eval()
    train_acc /= len(train_loader)
    train_loss /= len(train_loader)
    test_loss = 0
    test_acc = 0

    with torch.inference_mode():
        for X_test,y_test in test_loader:
            # Move data to the GPU
            X_test = X_test.to(device)
            y_test = y_test.to(device)

            # Forward pass
            y_test_logits = model(X_test.unsqueeze(1))

            # Calculate loss
            y_test = y_test.float().unsqueeze(1)
            test_batch_loss = loss_fn(y_test_logits, y_test)

            # Apply sigmoid and threshold to get binary predictions
            test_probs = torch.sigmoid(y_test_logits)
            test_batch_preds = (test_probs > 0.5).float()

            # Calculate accuracy using sklearn
            test_batch_acc = accuracy_score(y_test.cpu().numpy(), test_batch_preds.cpu().numpy())

            test_loss += test_batch_loss.item()
            test_acc += test_batch_acc

    test_loss /= len(test_loader)
    test_acc /= len(test_loader)

    print(f"Epoch: {epoch+1} | Train Loss: {train_loss:.4f} | Train ACC: {train_acc*100:.2f} | Test Loss: {test_loss:.4f} | Test ACC: {test_acc*100:.2f}")

print("Training complete.")





  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | Train Loss: 0.6326 | Train ACC: 73.00 | Test Loss: 0.4668 | Test ACC: 84.50
Epoch: 2 | Train Loss: 0.2111 | Train ACC: 95.00 | Test Loss: 0.3264 | Test ACC: 86.00
Epoch: 3 | Train Loss: 0.0370 | Train ACC: 99.61 | Test Loss: 0.3397 | Test ACC: 85.17
Epoch: 4 | Train Loss: 0.0107 | Train ACC: 100.00 | Test Loss: 0.3472 | Test ACC: 85.17
Epoch: 5 | Train Loss: 0.0050 | Train ACC: 100.00 | Test Loss: 0.3624 | Test ACC: 85.42
Training complete.


In [53]:
testing=['i absolutely loved product it exceeded expectation every way the quality topnotch work perfectly i highly recommend anyone looking reliable highquality option five star']
test_tensor=torch.tensor(tfidf.transform(testing).toarray(),dtype=torch.float)



In [54]:
test_tensor.shape

torch.Size([1, 34606])

In [55]:
with torch.inference_mode():
    out=model(test_tensor.unsqueeze(1).to(device))

In [58]:
print(f'predicted model {(torch.sigmoid(out)>0.5).int().item()} \n expected : 1 ')

predicted model 1 
 expected : 1 


In [52]:
preprocess(testing[0])

'i absolutely loved product it exceeded expectation every way the quality topnotch work perfectly i highly recommend anyone looking reliable highquality option five star'

In [57]:
test_tfidf.shape

(1200, 34606)